# Implementación optimizada y modular:
import re
import string
import unicodedata
from collections import Counter
from time import perf_counter
from typing import List, Tuple

def remove_punctuation(txt: str, method: str = 'regex', remove_underscore: bool = False) -> Tuple[str, float]:
    """Elimina signos de puntuación de `txt` y devuelve (texto_limpio, tiempo_segundos)."""
    start = perf_counter()
    if method == 'translate':
        # Muy rápido pero cubre solo puntuación ASCII (string.punctuation)
        trans = str.maketrans('', '', string.punctuation)
        clean = txt.lower().translate(trans)
        if remove_underscore:
            clean = clean.replace('_', '')
    elif method == 'regex':
        # Buen equilibrio: una pasada, soporta acentos vía re.UNICODE
        pattern = r'[^\w\s]|_' if remove_underscore else r'[^\w\s]'
        clean = re.sub(pattern, '', txt.lower(), flags=re.UNICODE)
    elif method == 'unicodedata':
        # Más preciso para todo Unicode, pero más lento (itera caracteres)
        clean = ''.join(ch for ch in txt if not unicodedata.category(ch).startswith('P'))
        clean = clean.lower()
        if remove_underscore:
            clean = clean.replace('_', '')
    else:
        raise ValueError('method must be one of: translate, regex, unicodedata')
    elapsed = perf_counter() - start
    return clean, elapsed

def process_text(text: str, n: int = 5, method: str = 'regex', remove_underscore: bool = False) -> Tuple[List[Tuple[str,int]], float]:
    """Normaliza, elimina puntuación, cuenta palabras y devuelve (top_n, tiempo_total)."""
    start = perf_counter()
    clean, t_clean = remove_punctuation(text, method=method, remove_underscore=remove_underscore)
    words = clean.split()
    top_n = Counter(words).most_common(n)
    elapsed = perf_counter() - start
    return top_n, elapsed

# Texto de ejemplo (con acentos para demostrar soporte Unicode)
text = '''In the heart of the city, Emily discovered a quaint little café, hidden away from the bustling streets.
The aroma of freshly baked pastries wafted through the air, drawing in passersby. As she sipped on her latte,
she noticed an old bookshelf filled with classics, creating a cozy atmosphere that made her lose track of time.'''

# Demostración: obtener top 5 usando regex (soporta acentos)
top5, time_total = process_text(text, n=5, method='regex')
print('Top 5:', top5)
print(f'Tiempo total (process_text): {time_total:.6f} s')

# Si quieres la medición separada de la limpieza:
clean_txt, time_clean = remove_punctuation(text, method='regex')
print(f'Tiempo limpieza (remove_punctuation): {time_clean:.6f} s')


## Ejercicio 1
### Optimización de código para procesamiento de texto

Se te ha entregado un código de procesamiento de texto que realiza las siguientes operaciones:

1. Convierte todo el texto a minúsculas.
2. Elimina los signos de puntuación.
3. Cuenta la frecuencia de cada palabra.
4. Muestra las 5 palabras mas comunes.

El código funciona, pero es ineficiente y puede optimizarse. Tu tarea es identificar las áreas que pueden ser mejoradas y reescribir esas partes para hacer el código mas eficiente y legible.


### Versión original (reemplazada)
La implementación original que usaba `replace` en un bucle fue reemplazada por la versión optimizada en la primera celda (arriba).
Usa las funciones `remove_punctuation` y `process_text` definidas en la celda de código superior para pruebas y mediciones.

In [8]:
import string
import time
import re
from collections import Counter
from time import perf_counter


def process_text(text):
    # Texto a minuscula
    text = text.lower()
    

    # Eliminación de puntuaciones
    for p in string.punctuation:
        text = text.replace(p, "")

    # Split text into words
    words = text.split()

    # Conteo de frecuencias
    frequencies = {}
    for w in words:
        if w in frequencies:
            frequencies[w] += 1
        else:
            frequencies[w] = 1

    sorted_frequencies = sorted(frequencies.items(), key = lambda x: x[1], reverse = True)

    # Obtener las 5 palabras más comunes
    top_5 = sorted_frequencies[:5]
    
    for w, frequency in top_5:
        print(f"'{w}': {frequency} times")

text = """
    In the heart of the city, Emily discovered a quaint little café, hidden away from the bustling streets. 
    The aroma of freshly baked pastries wafted through the air, drawing in passersby. As she sipped on her latte, 
    she noticed an old bookshelf filled with classics, creating a cozy atmosphere that made her lose track of time.
"""
t0 = perf_counter()
process_text(text)
t1 = perf_counter()
print("Tiempo:", t1 - t0)   

'the': 5 times
'of': 3 times
'in': 2 times
'a': 2 times
'she': 2 times
Tiempo: 0.0003122830003121635


Puntos a optimizar:

1. **Eliminar los signos de puntuación**: Usar `replace`  en un ciclo puede ser ineficiente, especialmente con textos largos. Busca una formas eficiente de eliminar los signos de puntuación.
2. **Contador de frecuencia**: El código verifica la existencia de cada palabra en el diccionario y luego actualiza su cuenta. Esto puede hacerse mas eficientemente con ciertas estructuras de datos en Python.
3. **Ordenar y seleccionar:** Considera si hay una forma mas directa o efectiva de obtener las 5 palabras mas frecuentes sin ordenar todas las palabras.
4. **Modularidad**: Divide el código en funciones mas pequeñas para que cada una puede realizar una tarea específica. Esto no solo optimizará el desempeño, sino también hará el código mas legible y mantenible.

In [12]:
# Versión sencilla (explicada como principiante)
import re
from collections import Counter
from time import perf_counter

text = """
    In the heart of the city, Emily discovered a quaint little café, hidden away from the bustling streets. 
    The aroma of freshly baked pastries wafted through the air, drawing in passersby. As she sipped on her latte, 
    she noticed an old bookshelf filled with classics, creating a cozy atmosphere that made her lose track of time.
"""

def remove_punctuation_simple(text):
    # 1) Paso para convertir texto a minusculas 
    text = text.lower()
    # 2) Eliminar signos de puntuación en una sola pasada con regex
    clean = re.sub(r'[^\w\s]', '', text, flags=re.UNICODE)
    return clean

"""[clean = re.sub(r'[^\w\s]', '', text, flags=re.UNICODE)]
1- re.sub(...) — significa “buscar y reemplazar” en una cadena. 
  Reemplaza lo que encuentra con otra cosa (aquí lo reemplaza por cadena vacía: lo borra).
2- r'[^\w\s]' — es la parte que dice qué borrar: 
 - Los corchetes [...] definen un conjunto de caracteres.
 - El ^ dentro de los corchetes quiere decir “todo lo que NO sea ...”.
 - \w significa “caracteres de palabra” (letras, números y el guion bajo _). En Python 3 incluye letras con acentos y muchas letras Unicode.
 - \s significa “espacios en blanco” (espacio, tab, salto de línea).
Entonces [^\w\s] = “cualquier cosa que NO sea letra/número/_ ni espacio”.
3- '' es la cadena con la que reemplazamos; como está vacía, eliminamos lo que coincide.
NOTA: flags=re.UNICODE — indica que la coincidencia debe respetar reglas Unicode (útil para letras con acento). 
En Python 3 normalmente ya está activado por defecto, pero no está de más ponerlo."""

def count_top_n_simple(text, n=5):
    # 3) Usamos Counter para contar rápido y most_common para obtener top n 
    words = text.split() # Convierte la cadena en una lista de palabras separadas por espacios
    counts = Counter(words) # Este metodo cuenta las ocurrencias de cada palabra
    return counts.most_common(n) # Este metodo devuelve las n palabras más comunes como lista de tuplas (palabra, conteo)

# AGRUPAMOS TODO EN UNA SOLA FUNCIÓN
def process_text_simple(text, n=5):
    # 4) Medimos tiempo de inicio con perf_counter y separamos responsabilidades
    t0 = perf_counter()
    clean = remove_punctuation_simple(text) # funcion 1 (convertir a minusculas y eliminar puntuacion)
    t1 = perf_counter() # medimos tiempo intermedio
    top = count_top_n_simple(clean, n) # funcion 2 (convertir a lista, contar palabras y obtener top n)
    t2 = perf_counter() # medimos tiempo final

    print('Texto limpio:', clean)
    print(f'Top {n}:, {top}\n')
    print(f'Tiempo limpieza: {t1 - t0:.6f} s')
    print(f'Tiempo conteo: {t2 - t1:.6f} s')
    print(f'Tiempo total: {t2 - t0:.6f} s')
    return top

# Prueba rápida con texto de ejemplo
process_text_simple(text, n=5)


Texto limpio: 
    in the heart of the city emily discovered a quaint little café hidden away from the bustling streets 
    the aroma of freshly baked pastries wafted through the air drawing in passersby as she sipped on her latte 
    she noticed an old bookshelf filled with classics creating a cozy atmosphere that made her lose track of time

Top 5:, [('the', 5), ('of', 3), ('in', 2), ('a', 2), ('she', 2)]

Tiempo limpieza: 0.000031 s
Tiempo conteo: 0.000053 s
Tiempo total: 0.000084 s


[('the', 5), ('of', 3), ('in', 2), ('a', 2), ('she', 2)]

## Ejercicio 2
### Optimización de código para procesamiento de listas

Se te ha dado el siguiente código que realiza operaciones en una lista de números para:

1. Filtrar los números pares.
2. Duplicar cada número.
3. Sumar todos los números.
4. Verificar si el resultado es un número primo.

El código entregado logra los objetivos, pero puede ser ineficiente. Tu tarea es identificar y mejorar las partes de ese código para mejorar su eficiencia.

Esta celda fue simplificada: la lógica y las mediciones se encuentran ahora en la primera celda de código.
Usa `process_text` para obtener el top-n y el tiempo de ejecución de forma fiable.

In [3]:
import math

def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(math.sqrt(n)) + 1):
        if n % i == 0:
            return False
    return True

def process_list(list_):
    filtered_list = []
    for num in list_:
        if num % 2 == 0:
            filtered_list.append(num)
    
    duplicate_list = []
    for num in filtered_list:
        duplicate_list.append(num * 2)
        
    sum = 0
    for num in duplicate_list:
        sum += num

    prime = is_prime(sum)
    
    return sum, prime

list_ = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result, result_prime = process_list(list_)
print(f"Result: {result}, ¿Prime? {'Yes' if result_prime else 'No'}")

Result: 60, ¿Prime? No


Puntos a optimizar:

1. **Filtrar las números**: El código recorre la lista original para filtrar los números pares. Considera una forma mas eficiente de filtrar la lista.
2. **Duplicación**: La lista es atravesada varias veces. ¿Hay alguna manera de hacer esto mas eficientemente?
3. **Suma**: Los números en la lista se suman a traves de un bucle. Python trae incluidas unas funciones que pueden optimizar esto.
4. **Función `is_prime`**: Aunque ésta función es relativamente eficiente, investiga si hay maneras de hacerla aun más rápida.
5. **Modularidad**: Considera dividir el código en funciones más pequeñas, cada una enfocada en una tarea específica.

In [ ]:
# TODO

Ambos ejercicios  ayudarán a mejorar tu habilidad de optimizar el desempeño del código y te darán un mejor entendimiento de como las diferentes estructuras de datos y técnicas de programación pueden afectar la eficiencia de tu código.